In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from itertools import islice

nltk.download('stopwords')

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

[nltk_data] Downloading package stopwords to /home/camix/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def llenar_nulls(df):
    df['titulo'] = df['titulo'].fillna('vacio')
    df['descripcion'] = df['descripcion'].fillna('vacio')
    return df

In [4]:
def generar_palabras_no_queridas(arr=None):
    '''Retorna un set con las palabras a excluir, arr, es un campo opcional para excluir tambien 
    las palabras en ese arreglo'''
    stop_words = set(stopwords.words('spanish'))
    puntuacion = [',', '.', '!', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '_', '\'', '\"', 
                  '(', ')', '[', ']', '{', '}', 'i', 'p', 'br', '<', '>', 'col.', '\\', '/', '$', 'm2', 
                  '..', '...', 'ii', '#', ';', ':', 'col']
    
    for x in puntuacion:
        stop_words.add(x)
    if arr:
        for x in arr:
            stop_words.add(x)
    return stop_words

def generar_token(textos):
    reviews = textos.str.cat(sep=' ')
    tokens = word_tokenize(reviews)
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]
    return tokens

In [5]:
def tf_idf (df,cols):
    cvec = CountVectorizer()
    tokens = generar_token(df[cols])
    cvec.fit(tokens)
    cvec_counts = cvec.fit_transform(tokens)
    transformer = TfidfVectorizer()
    transformed_weights = transformer.fit_transform(tokens)
    weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
    weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights}).set_index('term')
    return weights_df

In [6]:
def contador_mas_frecuentes(df, df_idf, col):
    contadores = []
    for texto in df[col].values:
        palabras_del_df = word_tokenize(texto)
        contador = 0
        for palabra in palabras_del_df:
            if palabra in df_idf.index:
                contador = contador + df_idf.loc[palabra].weight
        contadores.append(contador)
    return contadores

In [7]:
palabras_importantes = ['balcon', 'estacionamiento','niveles','pent', 'vista', 'panorámica','mar', 'lavanderia', 'patio', 'bien', 'hermosa', 'hermoso', 'jardin', 'vigilancia', 'preciosa', 'excelente', 'equipada', 'lujo', 'garage', 'piscina', 'pileta', 'privada', 'exclusiva', 'penthouse', 'facil', 'jacuzzi', 'seguridad', 'bodega']

def contador_mas_importantes(df, col):
    contadores = []
    for texto in df[col].values:
        palabras_del_df = word_tokenize(texto)
        contador = 0
        for palabra in palabras_del_df:
            if palabra in palabras_importantes:
                contador = contador + 1
        contadores.append(contador)
    return contadores

In [8]:
def features_textos(df):
    '''Devuelve una copia del dataframe con nuevas columnas, 
    con relacion a los textos'''
    
    df = df.copy()
    df = llenar_nulls(df)
   
    ''' Calculo el tf idf'''
    df_titulo = tf_idf(df, 'titulo')
    df_descripcion = tf_idf(df, 'descripcion')

    df['idf_titulo'] = contador_mas_frecuentes(df, df_titulo, 'titulo')
    df['idf_descripcion'] = contador_mas_frecuentes(df, df_descripcion, 'descripcion')
    
    df['peso_titulo'] = contador_mas_importantes(df, 'titulo')
    df['peso_descripcion'] = contador_mas_importantes(df, 'descripcion')
    
    return df

In [9]:
#df_tr = features_textos(df_train)

In [10]:
#df_te = features_textos(df_test)

In [1]:
# df_tr[['id', 'idf_titulo','idf_descripcion', 'peso_titulo', 'peso_descripcion']].to_csv('data/train_idf.csv', index=False)
# df_te[['id', 'idf_titulo','idf_descripcion', 'peso_titulo', 'peso_descripcion']].to_csv('data/test_idf.csv', index=False)